In [1]:

!pip install datasets

from datasets import load_dataset

dataset = load_dataset("SKNahin/bengali-transliteration-data")

dataset

  Using cached huggingface_hub-0.27.0-py3-none-any.whl.metadata (13 kB)
  Using cached aiohappyeyeballs-2.4.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached frozenlist-1.5.0-cp312-cp312-win_amd64.whl.metadata (14 kB)
  Using cached multidict-6.1.0-cp312-cp312-win_amd64.whl.metadata (5.1 kB)
  Using cached propcache-0.2.1-cp312-cp312-win_amd64.whl.metadata (9.5 kB)
  Using cached yarl-1.18.3-cp312-cp312-win_amd64.whl.metadata (71 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/480.6 kB ? eta -:--:--
    --------------------------------------- 10.2/480.6 kB ? eta -:--:--
    --------------------------------------- 10.2/480.6 kB ? eta -:--:--
   -- ------------------------------------ 30.7/480.6 kB 187.9 kB/s eta 0:00:03
   -- ------------------------------------ 30.7/480.6 kB 187.9 kB/s eta 0:00:03
   -- ------------------------------------ 30.7


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
C:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\datasets--SKNahin--bengali-transliteration-data. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingfa

DatasetDict({
    train: Dataset({
        features: ['bn', 'rm'],
        num_rows: 5006
    })
})

In [2]:
from datasets import load_dataset
dataset = load_dataset("SKNahin/bengali-transliteration-data")

# Splitting the dataset (80/20 split)
train_test_split = dataset['train'].train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']


In [3]:
val_dataset

Dataset({
    features: ['bn', 'rm'],
    num_rows: 1002
})

In [4]:
print(dataset['train'][0:5])

{'bn': ['স্ক্রোল করে ২০/৩০ সেকেন্ড এর ভিডিও পান নাই???', 'ও গুলা টরেন্ট সাইট এ পাবেন', 'ভক্কর চক্কর পোস্ট একটা করলেই এপ্রুভড.… নিশ্চই  ঘাবলা আছে', 'আমি টেস্ট করেই কোড দিছি…', 'এতো কষ্টের কি আছে সাকিবওয়াপ.টক,সাকিবওয়াপ.মল&এআইওভিডিওড্ল.মল থেকে সহজেই ডাউনলোড করা যায়'], 'rm': ['scroll kore 20/30 second er video pann nai???', 'o gula Torrent site e paben', 'vokkor chokkor post akta korlei approved…. nishchoi ghabla ache', 'ami test koreii code disi…', 'eto koster ki ache shakibwap.tk,shakibwap.ml&aiovideodl.ml theke shohojei downlod kora jay']}


In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi") # Example tokenizer

def tokenize_function(examples):
    inputs = examples["bn"]
    targets = examples["rm"]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/4004 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1002 [00:00<?, ? examples/s]

In [6]:
def filter_function(example):
    return len(example["bn"].split()) > 3 and len(example["rm"].split()) > 3

train_dataset = train_dataset.filter(filter_function)
val_dataset = val_dataset.filter(filter_function)


Filter:   0%|          | 0/4004 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1002 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")


In [8]:

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Changed from evaluation_strategy to eval_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    run_name="my_banglish_to_bangla_run"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer
)

trainer.train()
trainer.save_model()


NameError: name 'Seq2SeqTrainingArguments' is not defined